In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
# add project root folder to path to allow import local modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import local modules
from stochastic_models import *
from visualisations import *
from basic_estimator_model import *

In [2]:
N = 100
d = 6
T = 2

In [3]:
lambda_range = (1e-9 , 1e-3)
alpha_range = (8.3*1e-5, 0.83)
length_scale = np.sort(1/np.sqrt((2*alpha_range[0], 2*alpha_range[1])))

In [4]:
def reconstruct_alpha(length_scale):
    return 1/(2*(length_scale**2))
    

In [5]:
#Kernel Construction, doubt on the lambda hyperparameter
kernel = RBF(length_scale=random.uniform(length_scale[0] , length_scale[1]), length_scale_bounds=length_scale) + WhiteKernel(noise_level= 0.01, noise_level_bounds=lambda_range)


In [6]:
s = MaxCallStochasticModel(N,d,[1/12,11/12])
s.generate_samples()

In [7]:
y = s.y
X = s.X
S = s.S

V_T = s.generate_true_V(2)
V_0 = s.generate_true_V(0)


In [8]:
y

array([0.25978304, 0.23763887, 0.43337889, 0.46094563, 0.87153597,
       0.09361366, 0.        , 0.4309572 , 0.32586043, 0.80160838,
       0.18907219, 0.08180088, 0.10909007, 0.18162194, 0.26962045,
       0.67587341, 0.09322898, 0.54130324, 0.38554601, 0.38666782,
       0.4579794 , 0.        , 0.54443507, 0.52945426, 0.33349906,
       0.17923291, 0.22778009, 0.49995241, 0.07424929, 0.5080703 ,
       0.43434238, 0.05870709, 0.27431273, 0.52885416, 0.38380595,
       0.51293651, 0.32310197, 0.03519669, 0.14764892, 0.34920153,
       0.30164291, 0.33995962, 0.28323169, 0.16305493, 0.26680284,
       0.00990557, 0.28951967, 0.24116093, 0.36424334, 0.15915898,
       0.26984774, 0.31432057, 0.21854042, 0.156897  , 0.17628025,
       0.22376722, 0.20346308, 0.28619175, 0.33014464, 0.23572736,
       0.66268832, 0.24659388, 0.4548276 , 0.0734923 , 0.02958739,
       0.42299228, 0.63331829, 0.32958716, 0.12905389, 0.15442635,
       0.25618344, 0.60077124, 0.28793913, 0.15820418, 0.26243

In [9]:
def Flatten_Training_Sample(X , f):
    return np.array([f(x) for x in X])

In [10]:
#Shape of each sample j: [X_j[0,0],... ,X_j[d,0],...,X_j[0,T] , ..., X_j[d,T]]
Flatten_X_1 = Flatten_Training_Sample(X, lambda x : x.T.flatten())

#Shape of each sample j: [X_j[0,0],... ,X_j[0,T],...,X_j[d,0] , ..., X_j[d,T]]
Flatten_X_2 = Flatten_Training_Sample(X, lambda x : x.flatten())

In [11]:
#Model with Flatten_X_1
m_1 = EstimatorModelBase(kernel)
m_1.fit(Flatten_X_1,y)
m_1._predict_fX(Flatten_X_1)

In [12]:
optimal_hyperparameters_1 = (reconstruct_alpha(m_1.kernel.theta[0]), m_1.kernel.theta[1])
optimal_hyperparameters_1

(0.04484259135143556, -4.605170185988091)

In [13]:
#Model with Flatten_X_2
m_2 = EstimatorModelBase(kernel)
m_2.fit(Flatten_X_2, y)
m_2._predict_fX(Flatten_X_2)

In [14]:
optimal_hyperparameters_2 = (reconstruct_alpha(m_2.kernel.theta[0]), m_2.kernel.theta[1])
optimal_hyperparameters_2

(0.04484259135143556, -4.605170185988091)

In [17]:
def m_s(x, alpha, betha, d):
    return ((1+2*alpha)**(-d/2))*np.exp(((betha**2+4*alpha*betha - 2*alpha)*np.sum(x**2, axis=0)) / (4*alpha +2))

In [18]:
m_s(X[1,:,0],optimal_hyperparameters_2[0], 0, d)

0.6090239853534818

In [19]:
def w(x, d, T, gamma):
    return ((1-2*gamma)**(d*T))* np.exp(gamma*np.sum(x**2, axis=0))

In [20]:
def M_0(X): 
    M = np.zeros((X.shape[0], T))
    for j in range(len(M)):
        M[j, :]= np.array([m_s(x, optimal_hyperparameters_2[0], 0, d) for x in X[j,:,:].T ])
    return np.prod(M, axis=1)

In [21]:
def M_T(X, kernel):
    M = np.zeros(X.shape[0])
    for j in range(len(M)):
        M[j] = np.prod(kernel.diag(X[j]), axis=0)
    return M

In [22]:
#M_0(X)[j] == M_0(X(j)) in equation (25) for when V_X_t = V_X,0
#M_T(X)[j] == M_T(X(j)) in equation (25) for when V_X_t = V_X,T

In [23]:
M_0(X)

array([0.33345951, 0.35434256, 0.18245342, 0.32880349, 0.21465425,
       0.38442854, 0.30524447, 0.37135631, 0.29617148, 0.29912064,
       0.3490181 , 0.26852211, 0.4542835 , 0.31818107, 0.23860437,
       0.28464615, 0.42045694, 0.2831529 , 0.46200009, 0.35197423,
       0.2729724 , 0.43424498, 0.305462  , 0.41210191, 0.26683441,
       0.36882397, 0.25490257, 0.33079712, 0.42656821, 0.27160447,
       0.27312104, 0.48919946, 0.42695849, 0.26579258, 0.2420417 ,
       0.36773282, 0.40609384, 0.42612863, 0.33053044, 0.36242291,
       0.46345288, 0.3103611 , 0.36617654, 0.33485344, 0.33733104,
       0.39822963, 0.33852152, 0.37048608, 0.35217305, 0.3677558 ,
       0.37442568, 0.35869819, 0.43363261, 0.3701951 , 0.38234519,
       0.45949554, 0.42647197, 0.28969281, 0.38029346, 0.35791454,
       0.35439451, 0.36669335, 0.31859446, 0.52866633, 0.37783493,
       0.25357225, 0.39915893, 0.37189271, 0.34710263, 0.46595998,
       0.40710165, 0.22328234, 0.39549152, 0.42113024, 0.33741

In [24]:
M_T(X, kernel)

array([1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152015,
       1.06152015, 1.06152015, 1.06152015, 1.06152015, 1.06152

In [ ]:
#TODO
def recontruct_V_X (f_X, kernel, m_s, t):
        V_X = 